<a href="https://colab.research.google.com/github/skyloma20150818/img3/blob/master/ColabAndTpuTrain_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用 SSD MobileNet V2 FPNLite 训练目标检测模型
## 更适合移动端部署，精度更高

# 1. 安装 TensorFlow Object Detection API

In [ ]:
# 安装依赖 (使用 Colab 默认的 TensorFlow 版本)
# Colab 已预装 TensorFlow，无需重新安装
import tensorflow as tf
print(f'TensorFlow 版本: {tf.__version__}')

!pip install tf-models-official

# 克隆 TensorFlow Models
!git clone --depth 1 https://github.com/tensorflow/models.git

# 安装 Object Detection API
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install .

# 2. 挂载 Google Drive (保存训练进度)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 创建项目目录
import os
PROJECT_DIR = '/content/drive/MyDrive/poker_detection'
TRAINING_DIR = f'{PROJECT_DIR}/training'
EXPORT_DIR = f'{PROJECT_DIR}/exported_model'

os.makedirs(TRAINING_DIR, exist_ok=True)
os.makedirs(EXPORT_DIR, exist_ok=True)

print(f'训练目录: {TRAINING_DIR}')
print(f'导出目录: {EXPORT_DIR}')
print('\n✅ Google Drive 已挂载，训练进度会自动保存!')

# 3. 下载预训练模型 (SSD MobileNet V2 FPNLite 320x320)

In [ ]:
%cd /content

# 下载 SSD MobileNet V2 FPNLite 320x320 - 更适合移动端
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 或者使用 EfficientDet-Lite0 (更高精度)
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
# !tar -xvf efficientdet_d0_coco17_tpu-32.tar.gz

# 4. 准备数据集

In [ ]:
# 克隆你的数据集
!git clone https://github.com/skyloma20150818/img3.git

# 生成 CSV 和 TFRecord
%cd /content/models/research
!python object_detection/xml_to_csv.py --image_dir=/content/img3/

!mkdir -p /content/data
!python object_detection/generate_tfrecord.py \
    --csv_input=/content/img3/train_labels.csv \
    --image_dir=/content/img3/train \
    --output_path=/content/data/train.record

!python object_detection/generate_tfrecord.py \
    --csv_input=/content/img3/test_labels.csv \
    --image_dir=/content/img3/test \
    --output_path=/content/data/test.record

# 5. 创建配置文件

In [ ]:
%%writefile /content/ssd_mobilenet_v2_fpnlite_320.config
model {
  ssd {
    num_classes: 52  # 扑克牌 52 张
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 4e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.01
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.997
            scale: true
            epsilon: 0.001
          }
        }
        depth: 128
        num_layers_before_predictor: 4
        kernel_size: 3
        class_prediction_bias_init: -4.6
        share_prediction_tower: true
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 2
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.3
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
  }
}
train_config {
  batch_size: 16
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.08
          total_steps: 50000
          warmup_learning_rate: 0.026666
          warmup_steps: 1000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
  num_steps: 50000
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection"
  fine_tune_checkpoint_version: V2
}
train_input_reader {
  label_map_path: "/content/img3/data_label.pbtxt"
  tf_record_input_reader {
    input_path: "/content/data/train.record"
  }
}
eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: "/content/img3/data_label.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/data/test.record"
  }
}

# 6. 复制 label_map 文件

In [ ]:
# 使用仓库中的 data_label.pbtxt
!cp /content/img3/data_label.pbtxt /content/data/label_map.pbtxt

# 7. 开始训练 (保存到 Google Drive)

In [ ]:
%cd /content/models/research

# 训练模型 - checkpoint 保存到 Google Drive
!python object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/ssd_mobilenet_v2_fpnlite_320.config \
    --model_dir=/content/drive/MyDrive/poker_detection/training \
    --num_train_steps=50000 \
    --checkpoint_every_n=1000 \
    --alsologtostderr

# 8. 导出模型

In [ ]:
!python object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/ssd_mobilenet_v2_fpnlite_320.config \
    --trained_checkpoint_dir=/content/drive/MyDrive/poker_detection/training \
    --output_directory=/content/drive/MyDrive/poker_detection/exported_model

# 8. 转换为 TFLite (适合手机部署)

In [ ]:
import tensorflow as tf

# 加载 SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model('/content/exported_model/saved_model')

# 优化选项 - 量化以减小模型大小
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]  # FP16 量化

# 转换
tflite_model = converter.convert()

# 保存
with open('/content/model_fp16.tflite', 'wb') as f:
    f.write(tflite_model)

print('TFLite 模型已保存!')
print(f'模型大小: {len(tflite_model) / 1024 / 1024:.2f} MB')

# 9. INT8 量化 (更小的模型，适合低端手机)

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import glob

# 代表性数据集生成器 (用于 INT8 量化)
def representative_dataset():
    images = glob.glob('/content/img3/train/*.jpg')[:100]
    for img_path in images:
        img = Image.open(img_path).resize((320, 320))
        img = np.array(img, dtype=np.float32)
        img = np.expand_dims(img, axis=0)
        yield [img]

converter = tf.lite.TFLiteConverter.from_saved_model('/content/exported_model/saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_int8 = converter.convert()

with open('/content/model_int8.tflite', 'wb') as f:
    f.write(tflite_model_int8)

print('INT8 TFLite 模型已保存!')
print(f'模型大小: {len(tflite_model_int8) / 1024 / 1024:.2f} MB')

# 10. 下载模型

In [ ]:
from google.colab import files

# 下载 FP16 模型
files.download('/content/model_fp16.tflite')

# 下载 INT8 模型
files.download('/content/model_int8.tflite')